In [330]:
from IPython.display import Image
import requests
import pandas as pd
import openmeteo_requests
import requests_cache
from retry_requests import retry
from sqlalchemy import create_engine, text
from datetime import datetime, timedelta

In [331]:
def get_password_from_csv(filepath, key_type):
    df = pd.read_csv('/home/alfred/Documents/projects/api_keys - Sheet1.csv')
    password = df.loc[df['type'] == key_type, 'keys'].values[0]
    return password

# Get the password from the CSV file
password = get_password_from_csv('/path/to/keys.csv', 'database')

# Construct the database connection string
username = 'postgres'
host = 'localhost'
port = '5432'
database = 'postgres'
connection_string = f'postgresql+psycopg2://{username}:{password}@{host}/{database}'
engine = create_engine(connection_string)

In [332]:
# Get yesterday's date
last_week = (datetime.today() - timedelta(days=7)).strftime('%Y-%m-%d')

# Use yesterday's date in your logic
print(f"Fetching data up to {last_week}")


Fetching data up to 2024-06-09


In [333]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [334]:
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 33.44,
	"longitude": 112.07,
	"start_date": "2024-01-01",
	 "end_date": last_week,
	"hourly": ["temperature_2m", "rain", "weather_code", "cloud_cover", "is_day"],
	"temperature_unit": "fahrenheit"
}

In [335]:
responses = openmeteo.weather_api(url, params=params)

In [336]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°W")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 33.42706298828125°N 112.0271987915039°W
Elevation 1131.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [337]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_rain = hourly.Variables(1).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(2).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(3).ValuesAsNumpy()
hourly_is_day = hourly.Variables(4).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["rain"] = hourly_rain
hourly_data["weather_code"] = hourly_weather_code
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["is_day"] = hourly_is_day

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

                          date  temperature_2m  rain  weather_code  \
0    2024-01-01 00:00:00+00:00       23.371700   0.0           0.0   
1    2024-01-01 01:00:00+00:00       27.871700   0.0           0.0   
2    2024-01-01 02:00:00+00:00       31.381701   0.0           0.0   
3    2024-01-01 03:00:00+00:00       33.901699   0.0           0.0   
4    2024-01-01 04:00:00+00:00       35.611702   0.0           0.0   
...                        ...             ...   ...           ...   
3859 2024-06-09 19:00:00+00:00       62.431698   0.0           0.0   
3860 2024-06-09 20:00:00+00:00       61.441700   0.0           0.0   
3861 2024-06-09 21:00:00+00:00       60.721699   0.0           0.0   
3862 2024-06-09 22:00:00+00:00       61.441700   0.0           0.0   
3863 2024-06-09 23:00:00+00:00       65.581696   0.0           0.0   

      cloud_cover  is_day  
0        0.000000     1.0  
1        0.000000     1.0  
2        1.500000     1.0  
3       10.200000     1.0  
4        4.500000  

In [338]:
print(hourly_dataframe.head())

                       date  temperature_2m  rain  weather_code  cloud_cover  \
0 2024-01-01 00:00:00+00:00       23.371700   0.0           0.0          0.0   
1 2024-01-01 01:00:00+00:00       27.871700   0.0           0.0          0.0   
2 2024-01-01 02:00:00+00:00       31.381701   0.0           0.0          1.5   
3 2024-01-01 03:00:00+00:00       33.901699   0.0           0.0         10.2   
4 2024-01-01 04:00:00+00:00       35.611702   0.0           0.0          4.5   

   is_day  
0     1.0  
1     1.0  
2     1.0  
3     1.0  
4     1.0  


In [339]:
def fetch_data_with_sql_query(sql_query):
    """
    Fetch data from the specified API using an SQL query.

    :param sql_query: The SQL query to execute.
    :return: A list of records fetched from the API.
    """
    # URL for the API
    base_url = "https://www.phoenixopendata.com/api/3/action/datastore_search_sql"
    
    print(f"SQL Query: {sql_query}")  # Debug: Print the SQL query
    
    # Parameters for the API call
    params = {
        "sql": sql_query
    }
    
    # Make the API request
    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        print(f"API Response Content: {response.content[:1000]}")  # Debug: Print the first 1000 bytes of the API response content
        if 'result' in data and 'records' in data['result']:
            records = data['result']['records']
            return records
        else:
            print("Unexpected JSON structure.")
            return []
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return []

In [340]:
# Resource ID for Phoenix police data
phx_police_resource_id = "e8417a83-fcad-4cad-803c-423de3ad2d92"
start_date = "2023-01-01"

# Get the date a week ago
end_date = (datetime.today() - timedelta(days=7)).strftime('%Y-%m-%d')

# SQL query to fetch data from the specified date range
sql_query_phx_police = f'''
    SELECT * FROM "{phx_police_resource_id}"
    WHERE to_date("CALL_RECEIVED", 'MM/DD/YYYY HH12:MI:SSAM') >= '{start_date}'
    AND to_date("CALL_RECEIVED", 'MM/DD/YYYY HH12:MI:SSAM') <= '{end_date}'
'''

# Print the generated SQL query to verify
print(sql_query_phx_police)



    SELECT * FROM "e8417a83-fcad-4cad-803c-423de3ad2d92"
    WHERE to_date("CALL_RECEIVED", 'MM/DD/YYYY HH12:MI:SSAM') >= '2023-01-01'
    AND to_date("CALL_RECEIVED", 'MM/DD/YYYY HH12:MI:SSAM') <= '2024-06-09'



In [341]:
# Convert the data to a DataFrame
if phx_police_data:
    df_phx_police = pd.DataFrame(phx_police_data)
    print(df_phx_police.head(10))  # Display the first 10 rows
else:
    print("No data fetched.")

   _id                                         _full_text  INCIDENT_NUM  \
0    1  '00':9 '1/1/2024':7 '12':8 '202400000001':1 '2...  202400000001   
1    2  '00':12 '1/1/2024':10 '12':11 '202400000013':1...  202400000013   
2    3  '00':8 '1/1/2024':6 '12':7 '19th':12 '20240000...  202400000011   
3    4  '01':9 '01am':10 '1/1/2024':7 '12':8 '20240000...  202400000015   
4    5  '01':11 '1/1/2024':9 '12':10 '202400000012':1 ...  202400000012   
5    6  '02':11 '04am':12 '1/1/2024':9 '12':10 '202400...  202400000022   
6    7  '02':11 '1/1/2024':9 '12':10 '15am':12 '202400...  202400000024   
7    8  '02':9 '1/1/2024':7 '12':8 '202400000032':1 '3...  202400000032   
8    9  '03':10 '1/1/2024':8 '10am':11 '12':9 '2024000...  202400000023   
9   10  '03am':12 '04':11 '1/1/2024':9 '12':10 '202400...  202400000035   

  DISP_CODE          DISPOSITION FINAL_RADIO_CODE       FINAL_CALL_TYPE  \
0         R  DEPARTMENTAL REPORT              239                 FIGHT   
1         N   NO ACTION 

In [342]:
from datetime import datetime, timedelta

# Resource ID for Phoenix fire data
phx_fire_resource_id = "e832854c-6537-4223-ba26-674a7b799f49"
start_date = "2023-01-01T00:00:00"

# Get the date a week ago
end_date = (datetime.today() - timedelta(days=7)).strftime('%Y-%m-%dT23:59:59')

# SQL query to fetch data from the specified date range
sql_query_phx_fire = f'''
    SELECT * FROM "{phx_fire_resource_id}"
    WHERE "REPORTED" >= '{start_date}'
    AND "REPORTED" <= '{end_date}'
'''

# Print the generated SQL query to verify
print(sql_query_phx_fire)

# Fetch data using the SQL query
phx_fire_data = fetch_data_with_sql_query(sql_query_phx_fire)



    SELECT * FROM "e832854c-6537-4223-ba26-674a7b799f49"
    WHERE "REPORTED" >= '2023-01-01T00:00:00'
    AND "REPORTED" <= '2024-06-09T23:59:59'

SQL Query: 
    SELECT * FROM "e832854c-6537-4223-ba26-674a7b799f49"
    WHERE "REPORTED" >= '2023-01-01T00:00:00'
    AND "REPORTED" <= '2024-06-09T23:59:59'

API Response Content: b'{"help": "https://www.phoenixopendata.com/api/3/action/help_show?name=datastore_search_sql", "success": true, "result": {"sql": "\\n    SELECT * FROM \\"e832854c-6537-4223-ba26-674a7b799f49\\"\\n    WHERE \\"REPORTED\\" >= \'2023-01-01T00:00:00\'\\n    AND \\"REPORTED\\" <= \'2024-06-09T23:59:59\'\\n", "records_truncated": true, "records": [{"_id": 1, "_full_text": "\'-01\':3,4,9,10 \'00\':6 \'05\':13 \'15\':12 \'198xx\':14 \'2023\':2,8 \'23000002\':1 \'42\':7 \'8th\':16 \'check\':19 \'ckfout\':18 \'fire\':20,23 \'n\':15 \'out\':22 \'pl\':17 \'reported\':21 \'t00\':5,11", "INCIDENT": "23000002", "REPORTED": "2023-01-01T00:00:42", "CLOSED": "2023-01-01T00:15:0

In [343]:
# Convert fire data to DataFrame
df_phx_fire = pd.DataFrame(phx_fire_data)
print("fire DataFrame:")
print(df_phx_fire.head())

# Convert the data to a DataFrame
if phx_fire_data:
    df_phx_fire = pd.DataFrame(phx_fire_data)
    print(df_phx_fire.head(10))  # Display the first 10 rows
else:
    print("No data fetched.")

fire DataFrame:
   _id                                         _full_text  INCIDENT  \
0    1  '-01':3,4,9,10 '00':6 '05':13 '15':12 '198xx':...  23000002   
1    2  '-01':3,4,9,10 '05':6 '10':7 '12':12 '19th':15...  23000010   
2    3  '-01':3,4,9,10 '05':6 '148xx':14 '2023':2,8 '2...  23000011   
3    4  '-01':3,4,9,10 '07':6 '08':7 '2023':2,8 '22':1...  23000014   
4    5  '-01':3,4,9,10 '08':6 '10':12 '14':7 '2023':2,...  23000015   

              REPORTED               CLOSED         INCIDENT_ADDRESS  \
0  2023-01-01T00:00:42  2023-01-01T00:15:05           198XX N 8TH PL   
1  2023-01-01T00:05:10  2023-01-01T00:12:24    S 19TH AV/W DARREL RD   
2  2023-01-01T00:05:49  2023-01-01T00:27:30  148XX N BLACK CANYON AC   
3  2023-01-01T00:07:08  2023-01-01T00:22:54           41XX N 28TH AV   
4  2023-01-01T00:08:14  2023-01-01T01:10:56           32XX N 53RD LN   

  NATURE_CODE              NATURE_TEXT CATEGORY  
0      CKFOUT  check fire reported out     FIRE  
1        UNKF           

In [344]:
hourly_dataframe['date'] = pd.to_datetime(hourly_dataframe['date'], format='%Y-%m-%d %H:%M:%S%z')
df_phx_police['CALL_RECEIVED'] = pd.to_datetime(df_phx_police['CALL_RECEIVED'], format='%m/%d/%Y %I:%M:%S%p')
df_phx_fire['CLOSED'] = pd.to_datetime(df_phx_fire['CLOSED'], format='%Y-%m-%dT%H:%M:%S')
df_phx_fire['REPORTED'] = pd.to_datetime(df_phx_fire['REPORTED'], format='%Y-%m-%dT%H:%M:%S')

In [345]:
hourly_dataframe['date'] = hourly_dataframe['date'].dt.tz_localize(None)
df_phx_police['CALL_RECEIVED'] = df_phx_police['CALL_RECEIVED'].dt.tz_localize(None)
df_phx_fire['REPORTED'] = df_phx_fire['REPORTED'].dt.tz_localize(None)
df_phx_fire['CLOSED'] = df_phx_fire['CLOSED'].dt.tz_localize(None)

In [346]:
print(hourly_dataframe['date'].head())
print(df_phx_police['CALL_RECEIVED'].head())
print(df_phx_fire['REPORTED'].head())
print(df_phx_fire['CLOSED'].head())

0   2024-01-01 00:00:00
1   2024-01-01 01:00:00
2   2024-01-01 02:00:00
3   2024-01-01 03:00:00
4   2024-01-01 04:00:00
Name: date, dtype: datetime64[ns]
0   2024-01-01 00:00:26
1   2024-01-01 00:00:37
2   2024-01-01 00:00:41
3   2024-01-01 00:01:01
4   2024-01-01 00:01:43
Name: CALL_RECEIVED, dtype: datetime64[ns]
0   2023-01-01 00:00:42
1   2023-01-01 00:05:10
2   2023-01-01 00:05:49
3   2023-01-01 00:07:08
4   2023-01-01 00:08:14
Name: REPORTED, dtype: datetime64[ns]
0   2023-01-01 00:15:05
1   2023-01-01 00:12:24
2   2023-01-01 00:27:30
3   2023-01-01 00:22:54
4   2023-01-01 01:10:56
Name: CLOSED, dtype: datetime64[ns]


In [347]:
# Rename the column in the DataFrame
hourly_dataframe.rename(columns={'temperature_2m': 'temperature'}, inplace=True)





In [348]:
print(df_phx_police.columns.tolist())


['_id', '_full_text', 'INCIDENT_NUM', 'DISP_CODE', 'DISPOSITION', 'FINAL_RADIO_CODE', 'FINAL_CALL_TYPE', 'CALL_RECEIVED', 'HUNDREDBLOCKADDR', 'GRID']


In [349]:
df_phx_police.columns = df_phx_police.columns.str.strip()
print(df_phx_police.columns.tolist())


['_id', '_full_text', 'INCIDENT_NUM', 'DISP_CODE', 'DISPOSITION', 'FINAL_RADIO_CODE', 'FINAL_CALL_TYPE', 'CALL_RECEIVED', 'HUNDREDBLOCKADDR', 'GRID']


In [350]:
print(df_phx_fire.columns.tolist())

['_id', '_full_text', 'INCIDENT', 'REPORTED', 'CLOSED', 'INCIDENT_ADDRESS', 'NATURE_CODE', 'NATURE_TEXT', 'CATEGORY']


In [351]:
df_phx_fire.columns = df_phx_fire.columns.str.strip()
print(df_phx_fire.columns.tolist())

['_id', '_full_text', 'INCIDENT', 'REPORTED', 'CLOSED', 'INCIDENT_ADDRESS', 'NATURE_CODE', 'NATURE_TEXT', 'CATEGORY']


In [352]:
df_phx_police.drop(columns=['_id'], inplace=True)
df_phx_police.drop(columns=['_full_text'], inplace=True)
print(df_phx_police.columns.tolist())

['INCIDENT_NUM', 'DISP_CODE', 'DISPOSITION', 'FINAL_RADIO_CODE', 'FINAL_CALL_TYPE', 'CALL_RECEIVED', 'HUNDREDBLOCKADDR', 'GRID']


In [353]:
df_phx_fire.drop(columns=['_full_text'], inplace=True)
print(df_phx_police.columns.tolist())

['INCIDENT_NUM', 'DISP_CODE', 'DISPOSITION', 'FINAL_RADIO_CODE', 'FINAL_CALL_TYPE', 'CALL_RECEIVED', 'HUNDREDBLOCKADDR', 'GRID']


In [354]:
# Remove the _id column
#df_phx_police.drop(columns=['_id'], inplace=True)
df_phx_police.rename(columns={
    'INCIDENT_NUM': 'incident_num',
    'DISP_CODE': 'disp_code',
    'DISPOSITION': 'disposition',
    'FINAL_RADIO_CODE': 'final_radio_code',
    'FINAL_CALL_TYPE': 'final_call_type',
    'CALL_RECEIVED': 'call_received',
    'HUNDREDBLOCKADDR': 'hundredblockaddr',
    'GRID': 'grid'
}, inplace=True)


In [355]:
# Establish a connection
with engine.connect() as connection:
    # Begin a transaction
    with connection.begin():
        # Execute DELETE statements for multiple tables
        connection.execute(text("DELETE FROM phx_police"))
        connection.execute(text("DELETE FROM phx_fire"))
        connection.execute(text("DELETE FROM weather"))

In [356]:
# Print the relevant columns to inspect for None values
print("Inspecting critical columns for None values:")

# Print a sample of the columns to see their content
print(hourly_dataframe[['date', 'temperature', 'rain', 'weather_code', 'cloud_cover', 'is_day']].head(20))

# Print rows where any critical column has None values
none_values = hourly_dataframe[
    hourly_dataframe[['temperature', 'rain', 'weather_code', 'cloud_cover']].isnull().any(axis=1)
]

print("Rows with None values in critical columns:")
print(none_values)


Inspecting critical columns for None values:
                  date  temperature  rain  weather_code  cloud_cover  is_day
0  2024-01-01 00:00:00    23.371700   0.0           0.0     0.000000     1.0
1  2024-01-01 01:00:00    27.871700   0.0           0.0     0.000000     1.0
2  2024-01-01 02:00:00    31.381701   0.0           0.0     1.500000     1.0
3  2024-01-01 03:00:00    33.901699   0.0           0.0    10.200000     1.0
4  2024-01-01 04:00:00    35.611702   0.0           0.0     4.500000     1.0
5  2024-01-01 05:00:00    36.961700   0.0           0.0     2.400000     1.0
6  2024-01-01 06:00:00    38.221699   0.0           1.0    21.600000     1.0
7  2024-01-01 07:00:00    38.221699   0.0           2.0    63.600002     1.0
8  2024-01-01 08:00:00    36.421700   0.0           2.0    51.600002     1.0
9  2024-01-01 09:00:00    35.251701   0.0           0.0     6.000000     1.0
10 2024-01-01 10:00:00    31.561701   0.0           1.0    31.800001     0.0
11 2024-01-01 11:00:00    31.11

In [357]:
df_phx_police.to_sql('phx_police', con=engine, index=False, if_exists='append')
df_phx_fire.to_sql('phx_fire', con=engine, index=False, if_exists='append')
hourly_dataframe.to_sql('weather', con=engine, index=False, if_exists='append')

864

In [370]:
# Create temporary columns with truncated date
df_phx_police['call_received_hour'] = df_phx_police['call_received'].dt.floor('h')
hourly_dataframe['date_hour'] = hourly_dataframe['date'].dt.floor('h')

# Perform the left join
df_merged = pd.merge(df_phx_police, hourly_dataframe, how='left', left_on='call_received_hour', right_on='date_hour')

# Select the desired columns and drop temporary columns
df_phx_weather_police = df_merged[['incident_num', 'disposition', 'final_call_type', 'call_received', 'temperature', 'rain', 'weather_code', 'cloud_cover']]